# 1. Logistic Regression

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
# from config import db_password
from sqlalchemy import inspect
import matplotlib.pyplot as plt
import pandas as pd

business_df = pd.read_csv("../../../Data/business_census_merged_dataset.csv")


# Categorizing restaurants based on stars ratings
business_df["Category"] = pd.cut(business_df["Stars_Rating"],bins=[0.9,3.5,5],
                                 labels=["Lower Performance", "Higher Performance"])


def changeStatus(status):
    if status == "Lower Performance":
        return 0
    else:
        return 1
    
business_df['Category_Encoded'] = business_df["Category"].apply(changeStatus)
business_df["Category_Encoded"] = pd.to_numeric(business_df["Category_Encoded"])


business_df["Median_Income(dollars)"] = pd.to_numeric(business_df["Median_Income(dollars)"], errors='coerce')
business_df["Mean_Income(dollars)"] = pd.to_numeric(business_df["Mean_Income(dollars)"], errors='coerce')

business_df = business_df.dropna(subset=['Median_Income(dollars)'])

In [39]:
business_df

,Restaurant_ID,Restaurants_Name,Address,City,State,Postal_Code,Latitude,Longitude,Stars_Rating,Review_Count,...,Happy_Hour,Noise_Level,Dietary_Restrictions,Total_Estimate_Households_per_Zip,Total_Estimate_Married-couple_Family_households,Total_Estimate_Nonfamily_households,Median_Income(dollars),Mean_Income(dollars),Category,Category_Encoded
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0,1,0,7823,1743,5771,60676,91522,Higher Performance,1
1,WYYdQDjx-DsCanlP0DpImQ,J Sushi,243 S 10th St,Philadelphia,PA,19107,39.947033,-75.157434,4.5,72,...,0,4,0,7823,1743,5771,60676,91522,Higher Performance,1
2,ngvE1G9IckUO-ljZBFo23w,Mumbai Bistro,930 Locust St,Philadelphia,PA,19107,39.947207,-75.157252,4.0,217,...,0,1,0,7823,1743,5771,60676,91522,Higher Performance,1
3,IgNDlyLERBwFGphgcC5VDQ,Cosi,1128 Walnut St,Philadelphia,PA,19107,39.948751,-75.160449,2.5,63,...,0,4,0,7823,1743,5771,60676,91522,Lower Performance,0
4,L_sXNadtVHjxMw7Yhvkj9Q,Naoki Ramen,915 Race St,Philadelphia,PA,19107,39.955454,-75.154900,4.0,92,...,0,2,0,7823,1743,5771,60676,91522,Higher Performance,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24734,IFUgn1pFknZQ9OpN9EmyYw,Hudock's Frozen Custard Stand,544 Salem Quinton Rd,Salem,NJ,8079,39.560427,-75.448576,4.0,35,...,0,2,0,4018,1397,1574,42077,67275,Higher Performance,1
24735,tZy556-b1uO9pcy0WsumIA,Charlie's Restaurant & Catering,6129 E Saint Bernard Hwy,Violet,LA,70092,29.900428,-89.901686,4.0,71,...,0,2,0,2690,1417,624,44753,62162,Higher Performance,1
24736,jMStvE-tQzSpRCAO0nAE6g,Red Robin Gourmet Burgers and Brews,4605 S Landing Way,Tucson,AZ,32246,32.167311,-110.986422,3.0,159,...,1,4,0,21387,9033,8741,61730,76508,Lower Performance,0
24737,r4oV3EWnJX-1hjhNcKMI7w,Ortino's Northside,1355 Gravel Pike,Perkiomenville,PA,18074,40.291583,-75.481124,3.5,32,...,1,4,0,2298,1512,495,99333,132779,Lower Performance,0


In [38]:
y = business_df["Category"]
from collections import Counter
Counter(y)

Counter({'Higher Performance': 12794, 'Lower Performance': 11945})

In [17]:
business_df.columns

Index(['Restaurant_ID', 'Restaurants_Name', 'Address', 'City', 'State',
       'Postal_Code', 'Latitude', 'Longitude', 'Stars_Rating', 'Review_Count',
       'Restaurants_Delivery', 'Outdoor_Seating', 'Accepts_CreditCards',
       'Price_Range', 'Alcohol', 'Good_For_Kids', 'Reservations',
       'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions', 'Total_Estimate_Households_per_Zip',
       'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households', 'Median_Income(dollars)',
       'Mean_Income(dollars)', 'Category', 'Category_Encoded'],
      dtype='object')

In [25]:
X = business_df[['Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions',
                'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households',
                'Median_Income(dollars)', 'Total_Estimate_Households_per_Zip']]

In [26]:
# Define the target
y = business_df["Category_Encoded"]
y

0        1
1        1
2        1
3        0
4        1
        ..
24734    1
24735    1
24736    0
24737    0
24738    0
Name: Category_Encoded, Length: 24739, dtype: int64

In [27]:
# Split the model into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, 
                                                   random_state=1, 
                                                    stratify=y)

In [28]:
# Instantiate a Logistic Regression Model
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state=1)
model

LogisticRegression(random_state=1)

In [29]:
# Train the model
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [30]:
# Make predictions
y_pred = model.predict(X_test)

In [31]:
# Evaluate the Model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.5734842360549717

In [35]:
y_pred_train = model.predict(X_train)
accuracy_score(y_train, y_pred_train)

0.5866120513096906

In [36]:
# Calculate the confusion matrix 
from sklearn.metrics import confusion_matrix, classification_report

matrix=confusion_matrix(y_test, y_pred)
print(matrix)

[[1872 1114]
 [1524 1675]]


In [37]:
report=classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.55      0.63      0.59      2986
           1       0.60      0.52      0.56      3199

    accuracy                           0.57      6185
   macro avg       0.58      0.58      0.57      6185
weighted avg       0.58      0.57      0.57      6185

